In [1]:
import os
import sys
import numpy as np
import pandas as pd
from os.path import dirname
import math
import xmltodict
from urllib.request import urlopen
from xml.etree.ElementTree import parse
from fractions import Fraction
import piexif
from PIL import Image
from haversine import haversine,Unit
from datetime import time,timedelta
import matplotlib.pyplot as plt
%matplotlib inline

# automate initialflight

In [ ]:
root_path=os.walk("C:/Users/Manav/360CameraMicrophone/Testresults/20200701")
processed=[]
ind_mat=[]

In [ ]:
def find_match(a,b):
    print(a)
    return b in a

In [ ]:
def extract_time(str):
    return str.split(" ")[1]

In [ ]:
for root,dirs,files in root_path:
    if "_" in root:
        time=root.split("_")[1]
        if time in processed:
            continue
        hrs=int(time[:2])
        mins=int(time[2:4])
        if hrs>12:
            hrs-=12
            if hrs<10:
                hrs="0"+str(hrs)
            mod_time=hrs+":"+time[2:4]+"PM"
        else:
            mod_time=str(hrs)+":"+time[2:4]+"AM"
        print(root)
        print(mod_time)
        flware_dat=pd.read_excel(os.path.join(dirname(root),"Flightaware.xls"))
        print(flware)
        dep_time=np.array(list(flware_dat.iloc[:,3]))
        print(dep_time)
        vc_fun=np.vectorize(find_match)
        ind_true=vc_fun(dep_time,mod_time)
        print(ind_true)
        inds=np.where(vc_fun(dep_time,mod_time))
        print(inds[0])
        if len(inds[0])>0:
            print(inds)
            models=np.array(list(flware_dat.iloc[inds[0],1]))
            tails=np.array(list(flware_dat.iloc[inds[0],0]))
            print(models)
            print(tails)

In [ ]:
for root,dirs,files in root_path:
    for d in dirs:
        dir_path=os.walk(d)
        flware_dat=pd.read_excel(os.path.join(root,d,"Flightaware.xls"))
        for r,di,fil in dir_path:
            print(di)
            for d1 in di:
                print(d1)
                if "_" in d1:
                    if d1 not in processed:
                        processed.append(d1)
                        time=d1.split("_")[1]
                        hrs=int(time[:2])
                        mins=int(time[2:4])
                        if hrs>12:
                            hrs=hrs-12
                            mod_time=hrs+":"+time[2:4]+"PM"
                        else:
                            mod_time=hrs+":"+time[2:4]+"AM"
                        dep_time=np.array(list(flware_dat.iloc[:,3]))
                        inds=np.where(dep_time==mod_time)
                        models=np.array(list(flware_dat.iloc[inds,1]))
                        for y in models:
                            f1=y+time
                            print(pd.read_excel(os.path.join(root,d,f1,"\Initialflight.xls")))
                    else:
                        continue

## Automate directory

In [ ]:
root_path=os.walk("C:/Users/Manav/20200701")

In [ ]:
def convert_to_24h(s):
    hr=s[0:2]
    if len(s)==7:
        pm_or_am=s[5:7]
        if pm_or_am=='PM' and int(hr)!=12:
            hr=str(int(hr)+12)
            modified=str(hr)+s[3:5]
    else:
        pm_or_am=s[8:10]
        if pm_or_am=='PM' and int(hr)!=12:
            hr=str(int(hr)+12)
            modified=str(hr)+s[3:5]+s[6:8]
    return modified

In [ ]:
for root,dirs,files in root_path:
    print(root)
    if os.path.isfile(os.path.join(root,"Flightaware.xls"))==True:
        dat=pd.read_excel(os.path.join(root,"Flightaware.xls"),header=1)
        dat.dropna()
        print(dat)
        do_time=pd.Series(dat.iloc[:,3])
        times=do_time.apply(lambda str:str.split(" ")[1].split("\xa0")[0])
        new_times=times.apply(convert_to_hh_mm)
        models=pd.Series(dat.iloc[:,1])
        mod_time=pd.concat([models,new_times],axis=1)
        print(mod_time)
        mod_time['merged']=mod_time[mod_time.columns[0]]+"_"+mod_time[mod_time.columns[1]]
        merged=mod_time['merged'].to_numpy()
        for i in merged:
            if type(i)!=float and os.path.isdir(os.path.join(root,i))==False:
                os.mkdir(os.path.join(root,i))
    else:
        continue

# exif cam

In [ ]:
path="C:/Users/Manav/360CameraMicrophone/Testresults/20200623/"
dat=pd.read_excel(os.path.join(path,"Metar.xls"))

loc_ind=np.where(np.array(dat.iloc[:,0])=="Location")[0]
lat,long=dat.iloc[loc_ind,1].to_numpy()[0].split(",")
lat=float(lat)
long=float(long)

In [20]:
def degtodms(dd):
    is_positive = dd >= 0
    dd = abs(dd)
    minutes,seconds = divmod(dd*3600,60)
    degrees,minutes = divmod(minutes,60)
    degrees = degrees if is_positive else -degrees
    return (degrees,minutes,seconds)

In [ ]:
lat_deg,lat_min,lat_secs=degtodms(lat)
long_deg,long_min,long_secs=degtodms(long)

lat_deg_tup=(int(abs(lat_deg)),1)
lat_min_tup=(int(lat_min),1)
lat_sec_num=600*lat_secs
lat_sec_tup=(int(lat_sec_num),600)
long_deg_tup=(int(abs(long_deg)),1)
long_min_tup=(int(long_min),1)
long_sec_num=600*long_secs
long_sec_tup=(int(long_sec_num),600)

root_path=os.walk(path)

w='https://nationalmap.gov/epqs/pqs.php?x={0}&y={1}&units={2}&output={3}'.format(long,lat,"Feet","xml")
var_url = urlopen(w)
xmldoc = parse(var_url)
root=xmldoc.iterfind('Elevation_Query')
for u in root:
    elev=u.findtext('Elevation')
    
elev_rational=Fraction(str(elev))
for root,dirs,files in root_path:
    for f in files:
        if ".JPG" in f:
            if elev=='-1000000':
                print('invalid coordinates')
                break
            img=Image.open(os.path.join(root,f))
            exif_dic=piexif.load(img.info['exif'])
            exif_dic['GPS'][piexif.GPSIFD.GPSLatitude]=(lat_deg_tup,lat_min_tup,lat_sec_tup)
            exif_dic['GPS'][piexif.GPSIFD.GPSLatitudeRef]='N' if lat_deg>0 else 'S'
            exif_dic['GPS'][piexif.GPSIFD.GPSLongitude]=(long_deg_tup,long_min_tup,long_sec_tup)
            exif_dic['GPS'][piexif.GPSIFD.GPSLongitudeRef]='E' if long_deg>0 else 'W'
            exif_dic['GPS'][piexif.GPSIFD.GPSAltitude]=(elev_rational.numerator,elev_rational.denominator)
            exif_dic['GPS'][piexif.GPSIFD.GPSAltitudeRef]=0
            #print(exif_dic['GPS'])
            exif_dic_bytes=piexif.dump(exif_dic)
            img.save(os.path.join(root,f),format='jpeg',exif=exif_dic_bytes)

# exif dest

In [ ]:
path='C:/Users/Manav/Testresults/20200701/'
root_path=os.walk(path)
for root,dirs,files in root_path:
    if os.path.isfile(os.path.join(root,'Geolocations.xlsx')):
        geolocationsData=pd.read_excel(os.path.join(root,'Geolocations.xlsx'),header=2)
        is_present_rectangle_coord=0
        for file_number in range(len(geolocationsData['File Name'])):
            if ".JPG" in file:
                #print(file_count)
                img=Image.open(os.path.join(root,geolocationsData['File Name'].iloc[file_number]))
                exif_dic=piexif.load(img.info['exif'])
                altitude=geolocationsData['DestAltitude'].iloc[file_number]
                latitude=geolocationsData['DestLat'].iloc[file_number]
                longitude=geolocationsData['DestLong'].iloc[file_number]
                elevation=geolocationsData['DestElevation'].iloc[file_number]
                if 'DestBearingPositive' in geolocationsData.columns:
                    bearing=geolocationsData['DestBearingPositive'].iloc[file_number]
                else:
                    bearing=geolocationsData['DestBearing'].iloc[file_number]
                distance=geolocationsData['DestDistance'].iloc[file_number]
                if 'Top' in geolocationsData.columns:
                    top=geolocationsData['Top'].iloc[file_number]
                    bottom=geolocationsData['Bottom'].iloc[file_number]
                    left=geolocationsData['Left'].iloc[file_number]
                    right=geolocationsData['Right'].iloc[file_number]
                    is_present_rectangle_coord=1

                distance_kms=distance*0.305*0.001
                elevation_rational=int(elevation)
                altitude_rational_numerator=altitude*100
                distance_rational_numerator=int(distance_kms*100)
                elevation_rational_numerator=elevation_rational*100
                bearing_rational_numerator=bearing*100

                lat_deg,lat_min,lat_secs=degtodms(latitude)
                longitude_deg,longitude_min,longitude_secs=degtodms(longitude)
                lat_deg_tup=(int(abs(lat_deg)),1)
                lat_min_tup=(int(lat_min),1)
                lat_sec_num=600*lat_secs
                lat_sec_tup=(int(lat_sec_num),600)
                longitude_deg_tup=(int(abs(longitude_deg)),1)
                longitude_min_tup=(int(longitude_min),1)
                longitude_sec_num=600*longitude_secs
                longitude_sec_tup=(int(longitude_sec_num),600)

                exif_dic['GPS'][piexif.GPSIFD.GPSDestLatitude]=(lat_deg_tup,lat_min_tup,lat_sec_tup)
                exif_dic['GPS'][piexif.GPSIFD.GPSDestLatitudeRef]='N' if lat_deg>0 else 'S'
                exif_dic['GPS'][piexif.GPSIFD.GPSDestLongitude]=(longitude_deg_tup,longitude_min_tup,longitude_sec_tup)
                exif_dic['GPS'][piexif.GPSIFD.GPSDestLongitudeRef]='E' if longitude_deg>0 else 'W'
                exif_dic['GPS'][piexif.GPSIFD.GPSTrack]=(int(altitude_rational_numerator),100)   #this is used as there is no other suitable field
                exif_dic['GPS'][piexif.GPSIFD.GPSDestDistance]=(distance_rational_numerator,100)
                exif_dic['GPS'][piexif.GPSIFD.GPSDestDistanceRef]='K'
                exif_dic['GPS'][piexif.GPSIFD.GPSImgDirection]=(elevation_rational_numerator,100)
                exif_dic['GPS'][piexif.GPSIFD.GPSDestBearing]=(int(bearing_rational_numerator),100)
                exif_dic['GPS'][piexif.GPSIFD.GPSDestBearingRef]='T'
                if is_present_rectangle_coord==1:
                    exif_dic['0th'][piexif.ImageIFD.ActiveArea]=(np.ushort(top),np.ushort(left),np.ushort(bottom),np.ushort(right))

                exif_dic_bytes=piexif.dump(exif_dic)
                img.save(os.path.join(root,geolocationsData['File Name'].iloc[file_number]),format='jpeg',exif=exif_dic_bytes)

In [ ]:
dat=pd.read_excel('Geolocations (4).xlsx',header=2)

In [ ]:
dat_na=dat[pd.isna(dat['DestLat'])]

In [ ]:
k=dat[0:9]

In [ ]:
knowns=k['time']

In [ ]:
def obtain_nums(x):
    low=0
    high=len(knowns)-1
    mid=(low+high)//2
    while low<=high and high>=0:
        mid=(low+high)//2
        if low==high-1:
            return (low,high)
        elif knowns[mid]>x:
            high=mid
        elif knowns[mid]<x:
            low=mid

In [ ]:
bounds=dat_na['time'].apply(obtain_nums)

In [ ]:
req_ind=bounds.index

In [ ]:
lists=[]
heights=[]

In [ ]:
req_ind=bounds.index

In [ ]:
dat['time']=dat['time'].apply(lambda x:x.strftime('%r'))

In [ ]:
dat['time']=pd.to_timedelta(dat['time'])

In [ ]:
for i in range(len(req_ind)):
    u_ind=bounds.index[i]
    print(u_ind)
    p1_ind=bounds.iloc[i,][0]
    p2_ind=bounds.iloc[i,][1]
    lat=dat.iloc[p1_ind,2]+((dat.iloc[u_ind,1]-dat.iloc[p1_ind,1])/(dat.iloc[p2_ind,1]-dat.iloc[p1_ind,1]))*(dat.iloc[p2_ind,2]-dat.iloc[p1_ind,2])
    long=dat.iloc[p1_ind,3]+((dat.iloc[u_ind,1]-dat.iloc[p1_ind,1])/(dat.iloc[p2_ind,1]-dat.iloc[p1_ind,1]))*(dat.iloc[p2_ind,3]-dat.iloc[p1_ind,3])
    height=dat.iloc[p1_ind,4]+((dat.iloc[u_ind,1]-dat.iloc[p1_ind,1])/(dat.iloc[p2_ind,1]-dat.iloc[p1_ind,1]))*(dat.iloc[p2_ind,4]-dat.iloc[p1_ind,4])
    lists.append((lat,long,height))

In [ ]:
lists

In [ ]:
req_ind+4

In [ ]:
dat_na

In [ ]:
bounds

In [ ]:
dat_na

# image extraction